# Gross final consumption

Test how renewable fuels add up to gross final consumption within each sector, i.e. which variables (renewable fuels) I need to sum to match gross final consumption (GFC) in the sector for renewables (RA000).

Explore what do I miss to add up to computed bioenergy consumption.

[SIEC vocabulary](http://dd.eionet.europa.eu/vocabulary/eurostat/siec/) for fuels and [Energy balance vocabulary](http://dd.eionet.europa.eu/vocabulary/eurostat/nrg_bal/) for variables.

Based on the exploration below and for the sake of simplifying computed bioenergy consumption I work with general categories: Primary solid biofuels, Liquid biofuels, Biogases and Renewable municipal waste. Which means omitting Bioliquids, Charcoal and Blended biogases or their 'Sustainable' sub-categories, which – depending on the sector or a country – would make sense to include to match Gross final consumption for particular sector.

In [1]:
import os
import datetime
import pandas as pd

In [2]:
csv_output_dir = datetime.datetime.today().strftime('%Y-%m-%d')

if not os.path.exists(csv_output_dir):
    os.mkdir(csv_output_dir)

In [3]:
def tidy_shares(df, csv_name):
    """Tidy the DataFrame, return it and output to a csv file"""
    # Year as a variable 
    df = df.melt(['siec', 'nrg_bal', 'unit', 'geo'], var_name='year')

    # Moving variables in nrg_bal column in the source from rows to individual columns
    df.set_index(['geo', 'year', 'siec', 'unit', 'nrg_bal'], inplace=True)
    df = df.unstack()
    # https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns
    df.columns = df.columns.get_level_values(1)
    df.columns.name = None
    df = df.reset_index()
    # All fuels and variables for electricity but tidy
    df.to_csv(os.path.join(os.path.abspath(csv_output_dir), csv_name), decimal=',', index_label='id')
    return df

In [4]:
# Standard international energy product classification (SIEC)
# Create Dictionary from siec codes to human readable labels

siec_url = 'http://dd.eionet.europa.eu/vocabulary/eurostat/siec/csv'
siec = pd.read_csv(siec_url)
fuels_dict = {k: v for k, v in zip(siec['Notation'], siec['Label'])}

## Electricity

Confirm that sum of GEP_RED for individual fuel category adds up to GFC_ELC.

In [5]:
# The source tsv file is hybrid with tabs and commas as separators
el_url = 'https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nrg_ind_ured.tsv.gz'

el = pd.read_table(el_url, delimiter='[\t,]+')

el.rename(columns={'geo\\time': 'geo'}, inplace=True)
# Some column names have additional whitespace
el.rename(columns={str(year) + ' ': year for year in range(2004, 2019)}, inplace=True)
el.rename(columns={'2004': 2004}, inplace=True)

In [6]:
el = tidy_shares(el, 'el_tidy.csv')
el

,geo,year,siec,unit,GEP_NPUMP_MIX,GEP_RED,GFC_ELC,NMLPRD_MIX,NMLPRD_PURE
0,AL,2004,E7000,GWH,0.0,NaN,NaN,0.0,4256.8
1,AL,2004,R5110-5150_W6000RI,GWH,NaN,0.000,NaN,NaN,NaN
2,AL,2004,R5292P,GWH,NaN,0.000,NaN,NaN,NaN
3,AL,2004,R5292SB,GWH,NaN,0.000,NaN,NaN,NaN
4,AL,2004,R5292SP,GWH,NaN,0.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10795,XK,2018,RA410,GWH,NaN,0.000,NaN,NaN,NaN
10796,XK,2018,RA420,GWH,NaN,2.031,NaN,NaN,NaN
10797,XK,2018,RA500,GWH,NaN,0.000,NaN,NaN,NaN
10798,XK,2018,TOTAL,GWH,NaN,NaN,6053.316,NaN,NaN


In [7]:
# Unique fuels in the original table
{fuels_dict[code]: code for code in el['siec'].unique()}

{'Electricity': 'E7000',
 'Primary solid biofuels': 'R5110-5150_W6000RI',
 'Pure bioliquids': 'R5292P',
 'Sustainable blended bioliquids (only bio-part)': 'R5292SB',
 'Sustainable pure bioliquids': 'R5292SP',
 'Sustainable blended bioliquids (only bio-part) from non-food sources': 'R5293SB',
 'Sustainable pure bioliquids from non-food sources': 'R5293SP',
 'Biogases': 'R5300',
 'Blended biogases': 'R5300B',
 'Renewables and biofuels': 'RA000',
 'Hydro': 'RA100',
 'Geothermal': 'RA200',
 'Wind': 'RA300',
 'Solar thermal': 'RA410',
 'Solar photovoltaic': 'RA420',
 'Tide, wave, ocean': 'RA500',
 'Total': 'TOTAL',
 'Renewable municipal waste': 'W6210'}

In [8]:
# Czech republic
renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500',
    'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210',
    ]
res = el['siec'].str.fullmatch('|'.join(renewables_codes))
cz = el['geo'].str.fullmatch('CZ') 


el[res & cz & (el['year'] == 2018)]

,geo,year,siec,unit,GEP_NPUMP_MIX,GEP_RED,GFC_ELC,NMLPRD_MIX,NMLPRD_PURE
1873,CZ,2018,R5110-5150_W6000RI,GWH,NaN,2120.884,NaN,NaN,NaN
1879,CZ,2018,R5300,GWH,NaN,2607.205,NaN,NaN,NaN
1882,CZ,2018,RA100,GWH,NaN,2235.928,NaN,NaN,NaN
1883,CZ,2018,RA200,GWH,NaN,0.000,NaN,NaN,NaN
1884,CZ,2018,RA300,GWH,NaN,596.351,NaN,NaN,NaN
1885,CZ,2018,RA410,GWH,NaN,0.000,NaN,NaN,NaN
1886,CZ,2018,RA420,GWH,NaN,2358.881,NaN,NaN,NaN
1887,CZ,2018,RA500,GWH,NaN,0.000,NaN,NaN,NaN
1889,CZ,2018,W6210,GWH,NaN,100.189,NaN,NaN,NaN


In [9]:
el[(el['siec'] == 'RA000') & cz & (el['year'] == 2018)]

,geo,year,siec,unit,GEP_NPUMP_MIX,GEP_RED,GFC_ELC,NMLPRD_MIX,NMLPRD_PURE
1881,CZ,2018,RA000,GWH,NaN,10019.438,NaN,NaN,NaN


In [10]:
el[res & cz & (el['year'] == 2018)]['GEP_RED'].sum()

10019.438

In [11]:
el[(el['siec'] == 'RA000') & cz & (el['year'] == 2018)]['GEP_RED']

1881    10019.438
Name: GEP_RED, dtype: float64

In [12]:
# Test multiple countries

def test_sector(df, country, renewables_codes, var_to_sum, gfc_var):
    df = df.copy()
    geo = df['geo'].str.fullmatch(f'{country}') 
    res = df['siec'].str.fullmatch('|'.join(renewables_codes))
    df[res & geo & (el['year'] == 2018)]

    fuels_sum = df.loc[res & geo & (df['year'] == 2018), var_to_sum].sum().sum()

    gfc = df[(df['siec'] == 'RA000') & geo & (df['year'] == 2018)][gfc_var].values[0]

    print(country, ':', fuels_sum, 'vs.', gfc)

In [13]:
renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500', 
    'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210', 
    'R5300B', 'R5292SP',
    ]

countries = [
    'CZ', 'DK', 'AT', 'NL', 'SK', 'PL',
    'DE', 'UK', 'SE', 'EU27_2020', 'EU28',
    ]

for country in countries:
    test_sector(el, country, renewables_codes, 'GEP_RED', 'GEP_RED' )

CZ : 10019.438 vs. 10019.438
DK : 22226.942000000003 vs. 22226.942000000003
AT : 54032.681000000004 vs. 54032.682
NL : 18516.356 vs. 18516.356
SK : 6527.814 vs. 6527.815
PL : 22837.542 vs. 22837.542
DE : 223729.869 vs. 223729.86899999998
UK : 107276.442 vs. 107276.442
SE : 96744.281 vs. 96744.281
EU27_2020 : 942737.7800000001 vs. 942737.781
EU28 : 1050014.223 vs. 1050014.223


This would support the idea I should count also Sustainable pure bioliquids (R5292SP) and Blended biogases (R5300B) in the electricity sector.

## Heating and cooling

Confirm that GFC_HEAT_CL_E is a sum of FC_IND_OTH_E, GHP_RED and PPRD_RED for individual fuelv category. For bioenergy relevant just the first two.

In [14]:
hc_url = 'https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nrg_ind_urhcd.tsv.gz'

# hc = pd.read_table('nrg_ind_urhcd.tsv', delimiter='[\t,]+')
hc = pd.read_table(hc_url, delimiter='[\t,]+')

hc.rename(columns={'geo\\time': 'geo'}, inplace=True)
hc.rename(columns={str(year) + ' ': year for year in range(2004, 2019)}, inplace=True)
hc.rename(columns={'2004': 2004}, inplace=True)

for col in range(2004,2019):
    hc[col] = pd.to_numeric(hc[col], errors='coerce')

In [15]:
hc = tidy_shares(hc, 'hc_tidy.csv')
hc

,geo,year,siec,unit,BIOG_G_ADJSHR,BIOG_G_SHR,BIOG_G_TRA,FC_IND_OTH_E,GFC_HEAT_CL_E,GHP_RED,PPRD_RED
0,AL,2004,R5110-5150_W6000RI,KTOE,NaN,NaN,NaN,233.018,NaN,0.0,NaN
1,AL,2004,R5160,KTOE,NaN,NaN,NaN,0.000,NaN,NaN,NaN
2,AL,2004,R5292,KTOE,NaN,NaN,NaN,0.000,NaN,NaN,NaN
3,AL,2004,R5292P,KTOE,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,AL,2004,R5292S,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11995,XK,2018,RA200,KTOE,NaN,NaN,NaN,0.000,NaN,0.0,NaN
11996,XK,2018,RA410,KTOE,NaN,NaN,NaN,0.361,NaN,0.0,NaN
11997,XK,2018,RA600,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,0.0
11998,XK,2018,TOTAL,KTOE,NaN,NaN,NaN,NaN,647.961,NaN,NaN


In [16]:
# Unique fuels in the original table
{fuels_dict[code]: code for code in hc['siec'].unique()}

{'Primary solid biofuels': 'R5110-5150_W6000RI',
 'Charcoal': 'R5160',
 'Bioliquids': 'R5292',
 'Pure bioliquids': 'R5292P',
 'Sustainable bioliquids': 'R5292S',
 'Sustainable blended bioliquids (only bio-part)': 'R5292SB',
 'Sustainable pure bioliquids': 'R5292SP',
 'Sustainable bioliquids from non-food sources': 'R5293S',
 'Sustainable blended bioliquids (only bio-part) from non-food sources': 'R5293SB',
 'Sustainable pure bioliquids from non-food sources': 'R5293SP',
 'Biogases': 'R5300',
 'Blended biogases': 'R5300B',
 'Renewables and biofuels': 'RA000',
 'Geothermal': 'RA200',
 'Solar thermal': 'RA410',
 'Ambient heat (heat pumps)': 'RA600',
 'Total': 'TOTAL',
 'Renewable municipal waste': 'W6210'}

In [17]:
# Added RA600
renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500', 
    'RA600', 'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210',
    ]

res = hc['siec'].str.fullmatch('|'.join(renewables_codes))
cz = hc['geo'].str.fullmatch('CZ') 


hc.loc[res & cz & (hc['year'] == 2018), :]

,geo,year,siec,unit,BIOG_G_ADJSHR,BIOG_G_SHR,BIOG_G_TRA,FC_IND_OTH_E,GFC_HEAT_CL_E,GHP_RED,PPRD_RED
2080,CZ,2018,R5110-5150_W6000RI,KTOE,NaN,NaN,NaN,2323.972,NaN,161.939,NaN
2090,CZ,2018,R5300,KTOE,NaN,NaN,NaN,152.379,NaN,17.461,NaN
2095,CZ,2018,RA200,KTOE,NaN,NaN,NaN,0.000,NaN,0.000,NaN
2096,CZ,2018,RA410,KTOE,NaN,NaN,NaN,20.780,NaN,0.000,NaN
2097,CZ,2018,RA600,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,172.798
2099,CZ,2018,W6210,KTOE,NaN,NaN,NaN,22.796,NaN,40.165,NaN


In [18]:
hc.loc[(hc['siec'] == 'RA000') & cz & (hc['year'] == 2018), :]

,geo,year,siec,unit,BIOG_G_ADJSHR,BIOG_G_SHR,BIOG_G_TRA,FC_IND_OTH_E,GFC_HEAT_CL_E,GHP_RED,PPRD_RED
2094,CZ,2018,RA000,KTOE,NaN,NaN,NaN,NaN,2912.289,NaN,NaN


In [19]:
hc.loc[
    res & cz & (hc['year'] == 2018),
    ['FC_IND_OTH_E', 'GHP_RED', 'PPRD_RED']
    ].sum().sum()

2912.29

In [20]:
# Test multiple countries
renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500',
    'RA600', 'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210',
    'R5300B', 'R5292S', 'R5160',
    ]

for country in countries:
    test_sector(
        hc,
        country,
        renewables_codes,
        ['FC_IND_OTH_E', 'GHP_RED', 'PPRD_RED'],
        'GFC_HEAT_CL_E'
        )

CZ : 2912.29 vs. 2912.289
DK : 3564.1730000000002 vs. 3566.663
AT : 4547.403 vs. 4547.404
NL : 1629.368 vs. 1629.366
SK : 642.5319999999999 vs. 642.533
PL : 5586.959 vs. 5586.96
DE : 14874.232 vs. 14877.288999999999
UK : 4198.0160000000005 vs. 4198.016
SE : 9634.304 vs. 9634.305
EU27_2020 : 98603.025 vs. 98658.948
EU28 : 102801.04100000003 vs. 102856.964


This would support the idea to add also Blended biogases (R5300B), Sustainable bioliquids (R5292S) – which is different from electricity sector where it was sustainable and pure – and charcoal (R5160). In the EU and DE aggregation still small difference (hopefully due to rounding errors).

# Transport

Confirm that GFC_TRA_E_NMULTI is a sum of FC_TRA_OTH_E_RED, FC_TRA_RAIL_E_RED, FC_TRA_ROAD_E_RED for fuel category.

In [21]:
tr_url = 'https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nrg_ind_urtd.tsv.gz'

tr = pd.read_table(tr_url, delimiter='[\t,]+')

tr.rename(columns={'geo\\time': 'geo'}, inplace=True)
tr.rename(columns={str(year) + ' ': year for year in range(2004, 2019)}, inplace=True)
tr.rename(columns={'2004': 2004}, inplace=True)

for col in range(2004,2019):
    tr[col] = pd.to_numeric(tr[col], errors='coerce')

In [22]:
# Tidy it
tr = tidy_shares(tr, 'tr_tidy.csv')
tr

,geo,year,siec,unit,BIOFCON_LIM_TRA,BIOF_FS_SHR,FC_TRA_E_RED,FC_TRA_OTH_E_RED,FC_TRA_RAIL_E_RED,FC_TRA_ROAD_E_RED,GFC_TRA_E_MULTI,GFC_TRA_E_NMULTI,LIM_TT,NRG_MNBRFT_E,ST_MS_TRA
0,AL,2004,E7000,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,AL,2004,E7100,KTOE,NaN,NaN,NaN,0.722,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,AL,2004,E7200,KTOE,NaN,NaN,NaN,0.310,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,AL,2004,R5200,KTOE,NaN,NaN,NaN,0.000,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,AL,2004,R5200,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,XK,2018,R5300,KTOE,NaN,NaN,NaN,0.000,0.0,0.0,NaN,NaN,NaN,NaN,NaN
23996,XK,2018,RA000,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,NaN,NaN,0.0
23997,XK,2018,RHYD,KTOE,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23998,XK,2018,RSYNF,KTOE,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Unique fuels in the original table
{fuels_dict[code]: code for code in tr['siec'].unique()}

{'Electricity': 'E7000',
 'Electricity - renewable': 'E7100',
 'Electricity - non-renewable': 'E7200',
 'Liquid biofuels': 'R5200',
 'Sustainable biofuels': 'R5250S',
 'Sustainable biofuels Annex IX': 'R5251S',
 'Sustainable biofuels Annex IX, Part A': 'R5252S',
 'Sustainable biofuels Annex IX, Part A (a) - from algae': 'R5253S',
 'Sustainable biofuels Annex IX, Part A (b) - from biomass in mixed municipal waste': 'R5254S',
 'Sustainable biofuels Annex IX, Part A (c) - from biowaste': 'R5255S',
 'Sustainable biofuels Annex IX, Part A (d) - from biomass in industrial waste': 'R5256S',
 'Sustainable biofuels Annex IX, Part A (e) - from straw': 'R5257S',
 'Sustainable biofuels Annex IX, Part A (f) - from animal manure and sewage sludge': 'R5258S',
 'Sustainable biofuels Annex IX, Part A (g) - from palm oil effluent and empty palm fruit bunches': 'R5259S',
 'Sustainable biofuels Annex IX, Part A (h) - from tall oil pitch': 'R5260S',
 'Sustainable biofuels Annex IX, Part A (i) - from crude 

In [24]:
# Added RHYD, RSYNF, E7100
renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500',
    'RA600', 'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210',
    'RHYD', 'RSYNF', 'E7100',
    ]

In [25]:
res = tr['siec'].str.fullmatch('|'.join(renewables_codes))
cz = tr['geo'].str.fullmatch('CZ') 


tr.loc[res & cz & (tr['year'] == 2018), :]

,geo,year,siec,unit,BIOFCON_LIM_TRA,BIOF_FS_SHR,FC_TRA_E_RED,FC_TRA_OTH_E_RED,FC_TRA_RAIL_E_RED,FC_TRA_ROAD_E_RED,GFC_TRA_E_MULTI,GFC_TRA_E_NMULTI,LIM_TT,NRG_MNBRFT_E,ST_MS_TRA
4161,CZ,2018,E7100,KTOE,NaN,NaN,NaN,1.52,42.205,1.760,NaN,NaN,NaN,NaN,NaN
4163,CZ,2018,R5200,KTOE,308.708,NaN,NaN,0.00,0.000,308.708,NaN,NaN,453.288,NaN,NaN
4164,CZ,2018,R5200,PC,NaN,4.767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4195,CZ,2018,R5300,KTOE,NaN,NaN,NaN,0.00,0.000,0.000,NaN,NaN,NaN,NaN,NaN
4197,CZ,2018,RHYD,KTOE,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4198,CZ,2018,RSYNF,KTOE,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
tr.loc[(tr['siec'] == 'RA000') & cz & (tr['year'] == 2018), :]

,geo,year,siec,unit,BIOFCON_LIM_TRA,BIOF_FS_SHR,FC_TRA_E_RED,FC_TRA_OTH_E_RED,FC_TRA_RAIL_E_RED,FC_TRA_ROAD_E_RED,GFC_TRA_E_MULTI,GFC_TRA_E_NMULTI,LIM_TT,NRG_MNBRFT_E,ST_MS_TRA
4196,CZ,2018,RA000,KTOE,NaN,NaN,NaN,NaN,NaN,NaN,424.543,354.194,NaN,NaN,0.0


In [27]:
tr.loc[res & cz & (tr['year'] == 2018), ['FC_TRA_OTH_E_RED', 'FC_TRA_RAIL_E_RED', 'FC_TRA_ROAD_E_RED']]

,FC_TRA_OTH_E_RED,FC_TRA_RAIL_E_RED,FC_TRA_ROAD_E_RED
4161,1.52,42.205,1.760
4163,0.00,0.000,308.708
4164,NaN,NaN,NaN
4195,0.00,0.000,0.000
4197,NaN,NaN,NaN
4198,NaN,NaN,NaN


In [28]:
tr.loc[res & cz & (tr['year'] == 2018), ['FC_TRA_OTH_E_RED', 'FC_TRA_RAIL_E_RED', 'FC_TRA_ROAD_E_RED']].sum().sum()

354.193

In [29]:

renewables_codes = [
    'RA100', 'RA200', 'RA300', 'RA410', 'RA420', 'RA500', 'RA600',
    'R5110-5150_W6000RI', 'R5200', 'R5300', 'W6210',
    'RHYD', 'RSYNF', 'E7100',
    ]

for country in countries:
    test_sector(tr, country, renewables_codes=renewables_codes, var_to_sum=['FC_TRA_OTH_E_RED', 'FC_TRA_RAIL_E_RED', 'FC_TRA_ROAD_E_RED'], gfc_var='GFC_TRA_E_NMULTI',)

CZ : 354.193 vs. 354.194
DK : 237.297 vs. 236.672
AT : 675.935 vs. 664.53
NL : 564.862 vs. 562.398
SK : 160.664 vs. 160.664
PL : 1002.405 vs. 1002.405
DE : 3055.523 vs. 3021.757
UK : 1453.573 vs. 1439.611
SE : 1643.2590000000002 vs. 1643.259
EU27_2020 : 17205.777000000002 vs. 17131.334
EU28 : 18659.35 vs. 18570.944


I the transport sector, if I do not work with general Liquid biofuels (R5200), there aren't any Sustainable liquid biofuels in Czechia, which is a country I need for comparisons.

Unfortunatelly in this way I overshoot the GFC for most of the other countries (except CZ, PL, SE, SK)